## Imports

In [1]:
import pyearthtools.data

ModuleNotFoundError: No module named 'edit'

## Variables

In [9]:
doi = '2022-01-01T0000'
var = '2t'

# Transforms
`edit.data` provides an interface to apply transformations to any dataset at the time of loading. Commonly used transformations, like region cutting, masking, filtering, interpolation and more can be found at `edit.data.transforms`.

However, it is also possible for a user to define their own transform to apply.

### Default Transforms
All inbuilt DataIndexes use these transforms to force data into a standard format and for their own individual filtering's.

ERA5's base_transforms are shown below,

In [4]:
pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single').base_transforms

Transform Collection:
   ConformNaming                 Force Standard Dimension Names
   StandardCoordinates           Force Coordinates to have common design
   SetType                       Set variables to consistent type. Skip if TypeError
   NameDeviation                 Rename variables in Dataset
   VariableTrim                  Trim Dataset to given variables

### Using prebuilt Transforms
`6-RegionCutting.ipynb` shows how to use a prebuilt Transform for cutting a dataset to a region of interest.


### Defining Your Own Transform
As different project or use cases may require different Transforms to be applied to the data, there a couple of ways to define and use a custom Transform.

#### Function
The first & easiest way is to pass a function to the transform argument, either in DataIndex initialisation or retrieval call. 
This function will automatically be wrapped with a FunctionTransform and added to the TransformCollection.

This function must have the signature:
```python
def function(dataset) -> type(dataset)
```

In [7]:
def custom_transform(dataset):
    """This applies a custom Transform to mark the dataset"""
    dataset.attrs['Transform Mark'] = True
    return dataset

First, lets add it is a base_transform, to be applied to any data retrieval call using that DataIndex

In [8]:
pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single', transforms = custom_transform).base_transforms

Transform Collection:
   ConformNaming                 Force Standard Dimension Names
   StandardCoordinates           Force Coordinates to have common design
   SetType                       Set variables to consistent type. Skip if TypeError
   NameDeviation                 Rename variables in Dataset
   VariableTrim                  Trim Dataset to given variables
   FunctionTransform             Implementing: custom_transform: This applies a custom Transform to mark the dataset

Then, lets just pass it as a Transform to that one data retrieval call

In [11]:
pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single')(doi, transforms = custom_transform).attrs

{'Transform Mark': True}

#### Transform Class
For more complex Transforms, the Transform Class can be implemented.

A user must implement the `.apply(dataset)` function, and must have the same signature as above: 
```python
def apply(dataset) -> type(dataset)
```
It is also important to note, that these Transforms can be used independently just like a function.

In [12]:
class CustomTransform(pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.transform.Transform):
    """Custom Transform Class to mark the dataset"""
    def __init__(self, value):
        self.value = value
    def apply(self, dataset):
        dataset.attrs['Transform Mark'] = self.value
        return dataset

##### Using the CustomTransform

In [13]:
pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single', transforms = CustomTransform('Wow')).base_transforms

Transform Collection:
   ConformNaming                 Force Standard Dimension Names
   StandardCoordinates           Force Coordinates to have common design
   SetType                       Set variables to consistent type. Skip if TypeError
   NameDeviation                 Rename variables in Dataset
   VariableTrim                  Trim Dataset to given variables
   CustomTransform               Custom Transform Class to mark the dataset

In [14]:
pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single')(doi, transforms = CustomTransform('Wow')).attrs

{'Transform Mark': 'Wow'}

In [15]:
CustomTransform('Wow')(pyearthtoolsrthtoolsrthtoolsrthtoolsrthtools.data.archive.ERA5(var, level = 'single')(doi)).attrs

{'Transform Mark': 'Wow'}

# Transform Collections
To apply multiple Transforms, a TransformCollection can be made.

Adding any function or Transform to a Transform will automatically create a new TransformCollection.

When calling this Collection, each Transform is applied in order. This Collection also implements many of the common list functions, such as `pop`, `remove` & `append`

In [18]:
collection = CustomTransform('Wow') + CustomTransform('Wow_2')
collection

Transform Collection:
   CustomTransform               Custom Transform Class to mark the dataset
   CustomTransform               Custom Transform Class to mark the dataset

In [19]:
collection.pop()

Transform:
   CustomTransform               Custom Transform Class to mark the dataset

In [20]:
collection

Transform Collection:
   CustomTransform               Custom Transform Class to mark the dataset